In [2]:
import pandas as pd
import numpy as np 
from py2neo import Node,Relationship,Graph, database,NodeMatcher 

Database conecction

In [23]:
graph=Graph()# or Graph('bolt://neo4j@localhost:7687')

Crar nodos "Gene" a partir de red de referencia dorothea_final.csv

In [4]:
genes=pd.read_csv(r"C:\Users\espin\OneDrive\Escritorio\MCI\SCRIPTS\dorothea_final.csv", delimiter="\t",header=None)
TF=genes[0]
Gen=genes[1]
TF_list=[]
Gen_list=[]
for start in TF:
        if start not in TF_list:#si no existe el nodo then
                TF_list.append(start) # agregalo
for start in Gen:
        if start not in Gen_list:#si no existe el nodo then
                Gen_list.append(start)#agregalo

In [5]:
len(Gen_list),len(TF_list)

(20244, 1395)

Tenemos 20244 genes de los cuales 1395 son factores de transcripcion

In [13]:
for gen in Gen_list[]:
    #print("CREATE(:GENE{name:'"+gen+"'})")
    graph.run("Create(a:Gene{name:$name})",name=gen)

Crear reaciones "Regulates" entre los genes creados segun dorothea_final.csv usando tuplas [crear las tuplas se demora un poco]

In [16]:
#genes=pd.read_csv(r"C:\Users\espin\OneDrive\Escritorio\MCI\SCRIPTS\dorothea_final.csv", delimiter="\t",header=None)
#genes2=list(genes.to_records(index=False))

for tupla in genes2[]:
    #print("MATCH(a:Gene{name:",tupla[0],") MATCH(b:Gene{name:",tupla[1],") CREATE (a)-[:Regulates]->(b)")
    graph.run("MATCH(a:Gene{name:$name}) MATCH(b:Gene{name:$name1}) CREATE (a)-[:Regulates]->(b)",name=tupla[0],name1=tupla[1])


Con DisGeNet database, crear los nodos para todas las enfermedades conocidas y las relaciones a todos los genes existentes

In [47]:
GDAs=pd.read_csv(r"C:\Users\espin\OneDrive\Escritorio\MCI\SCRIPTS\GDAs_16_7_21.csv")
#GDAs.head()
GDAs_1=GDAs.head()
GDAs_1#datagrame first 5 rows og GDAs

,Gene,Gene_Full_Name,Disease,Disease_id,Score_gda
0,A1BG,alpha-1-B glycoprotein,Schizophrenia,C0036341,0.30
1,NAT2,N-acetyltransferase 2,Malignant neoplasm of urinary bladder,C0005684,0.40
2,NAT2,N-acetyltransferase 2,Bladder Neoplasm,C0005695,0.40
3,NAT2,N-acetyltransferase 2,Malignant neoplasm of breast,C0006142,0.40
4,NAT2,N-acetyltransferase 2,Coronary Arteriosclerosis,C0010054,0.31


In [93]:
GDAs.shape

(60478, 5)

In [98]:
Desease_list=[] 
Desease_id_list=[]
for i in range(0,len(GDAs)):
#for desease ,id_desease in GDAs_1[["Disease","Disease_id"]]:
    if GDAs["Disease"][i] not in Desease_list:#si no existe el nodo then
        Desease_list.append(GDAs["Disease"][i])
        Desease_id_list.append([GDAs["Disease"][i],GDAs["Disease_id"][i]]) # agregalo
print(Desease_id_list[1],len(Desease_id_list))

['Malignant neoplasm of urinary bladder', 'C0005684'] 9347


In [113]:
Desease_id_list[1:3]

[['Malignant neoplasm of urinary bladder', 'C0005684'],
 ['Bladder Neoplasm', 'C0005695']]

Crear nodos "Desease", con atributo "CIU", y crear relaciones "GDA", con atributo "score", entre "Gene" y "Desease"(recordar despues encontrar una forma de agrupar enfermedades)

In [102]:
for des in Desease_id_list: 
    #print("CREATE(:GENE{name:'"+gen+"'})")
    graph.run("Create(a:Desease{name:$name,CUI:$cui})",name=des[0],cui=des[1])

In [118]:
for i in range(0,len(GDAs)):
    graph.run("MATCH(a:Desease{name:$name}) MATCH(b:Gene{name:$name1}) CREATE(a)-[:GDA{GDA_score:$name2}]->(b)",name1=GDAs["Gene"][i],name=GDAs["Disease"][i],name2=GDAs["Score_gda"][i])

VALIDAR cuales son las relaciones gda que no se incorporaron

In [ ]:
# contar todos los tipos de relaciones
MATCH ()-[relationship]->() 
RETURN TYPE(relationship) AS type, COUNT(relationship) AS amount
ORDER BY amount DESC;
# Contar nodos por label
MATCH (n) 
RETURN DISTINCT count(labels(n)), labels(n);

testeo funcion .evaluate()

In [154]:
 b=graph.run("MATCH(b:Gene{name:$name1}) return b.name",name1=GDAs["Gene"][0]).evaluate()
 print(type(b),b)

<class 'str'> A1BG


In [159]:
Gene_not_found=[]
for i in range(0,len(GDAs["Gene"])):
    t=GDAs["Gene"][i]
    b=graph.run("MATCH(b:Gene{name:$name1}) return b.name",name1=t).evaluate()
    if b!=GDAs["Gene"][i]:
        Gene_not_found.append(t)#e n este caso incluye genes repetidos

In [162]:
Gene_not_found_no_rep=[]

for i in range(0,len(Gene_not_found)):
#for desease ,id_desease in GDAs_1[["Disease","Disease_id"]]:
    if Gene_not_found[i] not in Gene_not_found_no_rep:#si no existe el nodo then
        Gene_not_found_no_rep.append(Gene_not_found[i])
Gene_not_found_no_rep[:10],len(Gene_not_found_no_rep)

(['SNORD116-1',
  'CDKN2B-AS1',
  'SPG37',
  'SPG38',
  'MIR300',
  'MIR875',
  'MIR744',
  'MIR877',
  'MIR665',
  'MIR873'],
 658)

Contrastar los genes en dorothea con genes que no encontrados

In [161]:
Gen_list[:10]

['ATF7IP',
 'DYRK1A',
 'TLK1',
 'ZMYM4',
 'ABCC1',
 'ABCC6',
 'ABHD5',
 'ABT1',
 'AC002066.1',
 'AC002127.4']

In [163]:
for gen in Gene_not_found_no_rep:
    #print("CREATE(:GENE{name:'"+gen+"'})")
    graph.run("Create(a:Gene{name:$name})",name=gen)

In [19]:
e2s=pd.read_csv(r"C:\Users\espin\OneDrive\Escritorio\MCI\SCRIPTS\t2g2s 4-10-21.txt", sep="\t", index_col=None)


In [21]:
Gene_no_rep=[]

for i in range(0,len(e2s["external_gene_name"])):
#for desease ,id_desease in GDAs_1[["Disease","Disease_id"]]:
    if e2s["external_gene_name"][i] not in Gene_no_rep:#si no existe el nodo then
        Gene_no_rep.append(e2s["external_gene_name"][i])
Gene_no_rep[:10],len(Gene_no_rep)

NameError: name 'Gene_not_found_no_rep' is not defined

In [22]:
Gene_no_rep[:10],len(Gene_no_rep)

(['MT-TF',
  'MT-RNR1',
  'MT-TV',
  'MT-RNR2',
  'MT-TL1',
  'MT-ND1',
  'MT-TI',
  'MT-TQ',
  'MT-TM',
  'MT-ND2'],
 61102)

In [26]:
Gene_not_found=[]
for i in range(0,len(Gene_no_rep)):
    t=Gene_no_rep[i]
    b=graph.run("MATCH(b:Gene{name:$name1}) return b.name",name1=t).evaluate()
    if b!=Gene_no_rep[i]:
        Gene_not_found.append(t)#en este caso incluye genes repetidos
Gene_not_found[:10],len(Gene_not_found)

(['MT-TF',
  'MT-RNR1',
  'MT-TV',
  'MT-RNR2',
  'MT-TL1',
  'MT-TI',
  'MT-TQ',
  'MT-TM',
  'MT-TW',
  'MT-TA'],
 41393)